In [1]:
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(plyr)
library(purrr)
library(vegan)
#library(arrow)
library(parallel)
library(ComplexHeatmap)
library(ggsci)
library(tidyverse)

------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: ‘plyr’


The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



Attaching package: ‘purrr’


The following object is masked from ‘package:plyr’:

    compact


Loading required package: permute

Loading required package: lattice

This is vegan 2.6-4

Loading required package: grid

ComplexHeatmap version 2.16.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cit

# Create nDEGs plot for each celltype (p <= 0.1)

In [2]:
master <- read.csv("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/Figure_Notebooks/Intermediate_files/master_combined_DEG_list_all_celltypes_forfigs.csv")

In [3]:
library(dplyr)
library(tidyr)

# Create all possible combinations of celltype, Direction, and type
all_combinations <- expand.grid(celltype = unique(master$celltype),
                                Direction = unique(master$Direction),
                                type = unique(master$type))

# Summarize the data
master_summary <- master %>%
  dplyr::mutate(Expression = case_when(log2FoldChange >= 0.1 & padj <= 0.1 ~ "BR2",
                                 log2FoldChange <= -0.1 & padj <= 0.1 ~ "BR1",
                                 TRUE ~ "Unchanged")) %>%
  dplyr::filter(!Expression == 'Unchanged') %>%
  dplyr::group_by(celltype, Direction, type) %>%
  dplyr::summarise(count = n(), .groups = 'drop')

# Left join with all_combinations and fill missing counts with 0
master_complete <- left_join(all_combinations, master_summary, by = c("celltype", "Direction", "type")) %>%
  dplyr::mutate(count = replace_na(count, 0)) %>% 
  filter(!is.na(Direction)) # Remove rows with NA in the Direction column



# Check the number of unique celltypes
length(unique(master_complete$celltype))


head(master_complete)
#all_combinations


[1] 71

,celltype,Direction,type,count
,<chr>,<chr>,<chr>,<int>
1,Activated memory B cell,BR1,DEGs_Male,0
2,Adaptive NK cell,BR1,DEGs_Male,0
3,ASDC,BR1,DEGs_Male,0
4,BaEoMaP cell,BR1,DEGs_Male,0
5,C1Q+ CD16 monocyte,BR1,DEGs_Male,1
6,CD14+ cDC2,BR1,DEGs_Male,0


In [4]:
dim(master_complete)

[1] 284   4

In [5]:
write.csv(master_complete, "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/Figure_Notebooks/Intermediate_files/nDEGs_counts_sourcedata.csv")